# Process Airbnb data

## Import neccessary packages, modules and credentials

In [9]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from credentials import host, port, user, password, dbname_raw, dbname_processed, table_name

## Ping Database to test connection

In [10]:
def ping_db(host, port, dbname, user, password):
    """
    Test connection to the PostgreSQL database
    """
    try:
        # Create a connection
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        # Create a cursor object
        cur = conn.cursor()
        # Execute a simple query
        cur.execute("SELECT 1")
        # Close the cursor and connection
        cur.close()
        conn.close()
        print("Successfully connected to the database.")
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return False 

ping_db(host, port, dbname_raw, user, password)

Successfully connected to the database.


## Fetch data and column names from the database

In [11]:
def fetch_data_and_columns(host, port, dbname, user, password, table_name):
    """
    Fetch data and column names from a PostgreSQL database
    """
    try:
        # Create a connection
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )

        # Create a cursor object
        cur = conn.cursor()

        # Execute a query to fetch the data
        cur.execute(f"SELECT * FROM {table_name}")

        # Fetch the data
        rows = cur.fetchall()

        # Fetch the column names
        column_names = [desc[0] for desc in cur.description]

        # Close the cursor and connection
        cur.close()
        conn.close()

        return rows, column_names
    except Exception as e:
        print(f"Error fetching data from the database: {e}")
        return None, None

# Fetch the data and column names
rows, column_names = fetch_data_and_columns(host, port, dbname_raw, user, password, table_name)

# Create the DataFrame
df = pd.DataFrame(rows, columns=column_names)

## Drop irrelevant columns

In [12]:
# These rows were deemed irrelevant for the analysis based on the data dictionary
columns_to_delete = ['scrape_id', 'host_thumbnail_url', 'host_picture_url', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'calendar_last_scraped', 'neighbourhood_group_cleansed', 'license']
filtered_df = df.drop(columns_to_delete, axis=1)

## Check authenticity of each row

In [13]:
print(f"Number of rows with empty ID: {filtered_df['id'].isnull().sum()}")
print(f"Number of rows with empty URL: {filtered_df['listing_url'].isnull().sum()}")
print(f"Number of valid picture URLs: {filtered_df['picture_url'].str.startswith('https').value_counts()}")

Number of rows with empty ID: 0
Number of rows with empty URL: 0
Number of valid picture URLs: picture_url
True    12495
Name: count, dtype: int64


## Format data

In [14]:
# Refer to the data dictionary and report to follow these data cleaning steps.
filtered_df['last_scraped'] = pd.to_datetime(filtered_df['last_scraped'])
filtered_df['description'] = filtered_df['description'].str.replace(r"[^a-zA-Z\d\_]+", " ")
filtered_df['host_since'] = pd.to_datetime(filtered_df['host_since'])
filtered_df['host_is_superhost'] = filtered_df['host_is_superhost'].fillna('f')
filtered_df['host_is_superhost'] = filtered_df['host_is_superhost'].map({'t': True, 'f': False}).astype('boolean')
filtered_df['bathrooms'] = filtered_df['bathrooms_text'].str.extract(r'^(\d+\.?\d*)')
filtered_df['price'] = filtered_df['price'].str.removeprefix('$').str.replace(',', '').astype(float)
filtered_df['maximum_nights'] = filtered_df['maximum_nights'].astype(int)
filtered_df['minimum_nights'] = filtered_df['minimum_nights'].astype(int)
filtered_df['number_of_reviews'] = filtered_df['number_of_reviews'].astype(int)
filtered_df['number_of_reviews_ltm'] = filtered_df['number_of_reviews_ltm'].astype(int)
filtered_df['number_of_reviews_l30d'] = filtered_df['number_of_reviews_l30d'].astype(int)
filtered_df['first_review'] = pd.to_datetime(filtered_df['first_review'])
filtered_df['last_review'] = pd.to_datetime(filtered_df['last_review'])
filtered_df['review_scores_rating'] = filtered_df['review_scores_rating'].astype(float).round(1)
filtered_df['review_scores_accuracy'] = filtered_df['review_scores_accuracy'].astype(float).round(1)
filtered_df['review_scores_cleanliness'] = filtered_df['review_scores_cleanliness'].astype(float).round(1)
filtered_df['review_scores_checkin'] = filtered_df['review_scores_checkin'].astype(float).round(1)
filtered_df['review_scores_communication'] = filtered_df['review_scores_communication'].astype(float).round(1)
filtered_df['review_scores_location'] = filtered_df['review_scores_location'].astype(float).round(1)
filtered_df['review_scores_value'] = filtered_df['review_scores_value'].astype(float).round(1)

## Validate correct formatting

In [15]:
filtered_df.info()
filtered_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12495 entries, 0 to 12494
Data columns (total 62 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            12495 non-null  int64         
 1   listing_url                                   12495 non-null  object        
 2   last_scraped                                  12495 non-null  datetime64[ns]
 3   name                                          12495 non-null  object        
 4   description                                   12156 non-null  object        
 5   neighborhood_overview                         6461 non-null   object        
 6   picture_url                                   12495 non-null  object        
 7   host_id                                       12495 non-null  int64         
 8   host_url                                      12495 non-null  obje

,id,listing_url,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,API_Neighborhood
0,6983,https://www.airbnb.com/rooms/6983,2022-06-25,Copenhagen 'N Livin',Lovely apartment located in the hip Nørrebro a...,"Nice bars and cozy cafes just minutes away, ye...",https://a0.muscache.com/pictures/42044170/f63c...,16774,https://www.airbnb.com/users/show/16774,Simon,...,4.9,4.7,4.7,f,1,1,0,0,1.10,København N
1,26057,https://www.airbnb.com/rooms/26057,2022-06-25,Lovely house - most attractive area,Our lovely house in the center of the city is ...,The neighborhood is the most famous one and th...,https://a0.muscache.com/pictures/miso/Hosting-...,109777,https://www.airbnb.com/users/show/109777,Kari,...,4.8,4.9,4.8,f,1,1,0,0,0.56,København Ø
2,26473,https://www.airbnb.com/rooms/26473,2022-06-25,City Centre Townhouse Sleeps 1-10 persons,GUESTHOUSE ON KNABROSTRÆDE - located in the ab...,None,https://a0.muscache.com/pictures/8e132ba0-b68c...,112210,https://www.airbnb.com/users/show/112210,Julia,...,4.7,4.9,4.6,f,11,6,5,0,2.06,København K
3,29118,https://www.airbnb.com/rooms/29118,2022-06-25,Best Location in Cool Istedgade,<b>The space</b><br />The apartment is situate...,None,https://a0.muscache.com/pictures/b83c7003-367a...,125230,https://www.airbnb.com/users/show/125230,Nana,...,5.0,4.8,4.8,f,1,1,0,0,0.15,København V
4,130253,https://www.airbnb.com/rooms/130253,2022-06-25,Lovely big room in Copenhagen Ø,"<b>The space</b><br />Big room (7x3,5 meters),...",None,https://a0.muscache.com/pictures/15048292/a6a0...,641583,https://www.airbnb.com/users/show/641583,Mathilde,...,4.9,4.7,4.6,f,2,0,2,0,0.16,København Ø


## Export data to processed database

In [16]:
try:
    # Create a SQLAlchemy engine
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname_processed}')

    # Write the DataFrame to the database table
    filtered_df.to_sql(table_name, engine, if_exists='replace', index=False)

    # Confirm the data has been written to the raw database
    print("Data has been written to the processed database.")
except Exception as e:
    print(f"Error writing data to the database: {e}")

Data has been written to the processed database.
